In [43]:
import xarray as xr 
import iris
import matplotlib.pyplot as plt 
import numpy as np 

In [44]:
# ds = xr.open_mfdataset('/home/links/ct715/eddy_feedback/daily_datasets/jra55_djf_ubar_ep.nc', 
#                        parallel=True, chunks={'time':31})

ds = xr.open_mfdataset('/home/links/ct715/eddy_feedback/daily_datasets/era5daily_djf_uvt_ubar_ep.nc', 
                        parallel=True, chunks={'time':31})

ds

<xarray.Dataset>
Dimensions:  (time: 4030, lon: 360, lat: 181, level: 37)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01T09:00:00 ... 2023-02-28T09:00:00
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * lat      (lat) float32 90.0 89.0 88.0 87.0 86.0 ... -87.0 -88.0 -89.0 -90.0
  * level    (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Data variables:
    t        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 181, 360), meta=np.ndarray>
    u        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 181, 360), meta=np.ndarray>
    v        (time, level, lat, lon) float32 dask.array<chunksize=(31, 37, 181, 360), meta=np.ndarray>
    ubar     (time, level, lat) float32 dask.array<chunksize=(31, 37, 181), meta=np.ndarray>
    ep1      (time, level, lat) float64 dask.array<chunksize=(31, 37, 181), meta=np.ndarray>
    ep2      (time, level, lat) float64 dask.array<chunksize=(31, 37, 181), meta=np.ndarray>
    div1     (time, level, lat) float64 dask.array<chunksize=(31, 37, 181), meta=np.ndarray>
    div2     (time, level, lat) float64 dask.array<chunksize=(31, 37, 181), meta=np.ndarray>

In [45]:
ds.level

<xarray.DataArray 'level' (level: 37)>
array([   1.,    2.,    3.,    5.,    7.,   10.,   20.,   30.,   50.,   70.,
        100.,  125.,  150.,  175.,  200.,  225.,  250.,  300.,  350.,  400.,
        450.,  500.,  550.,  600.,  650.,  700.,  750.,  775.,  800.,  825.,
        850.,  875.,  900.,  925.,  950.,  975., 1000.])
Coordinates:
  * level    (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
Attributes:
    standard_name:  air_pressure
    long_name:      pressure_level
    units:          millibars
    positive:       down
    axis:           Z

In [46]:
ubar = ds.ubar.to_iris()
div1 = ds.div1.to_iris()

In [47]:
import efp_saffin as saff 

In [48]:
import datetime as dt 
span = dt.timedelta(days=28)

ubar = saff.seasonal_mean(ubar, span)
div1 = saff.seasonal_mean(div1, span)

In [49]:
ubar = ubar.extract(iris.Constraint(air_pressure=500.))
div1 = div1.extract(iris.Constraint(air_pressure=500.))

# ubar = ubar.extract(iris.Constraint(level=500.))
# div1 = div1.extract(iris.Constraint(level=500.))

In [50]:
efp, efp_array = saff.eddy_feedback_parameter(div1, ubar)

/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2245: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'season_year', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'season_year'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2245: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check

In [51]:
efp.data 

array(0.32050844)

In [52]:
efp_array.data

efp_array.coord('latitude').points

array([72., 71., 70., 69., 68., 67., 66., 65., 64., 63., 62., 61., 60.,
       59., 58., 57., 56., 55., 54., 53., 52., 51., 50., 49., 48., 47.,
       46., 45., 44., 43., 42., 41., 40., 39., 38., 37., 36., 35., 34.,
       33., 32., 31., 30., 29., 28., 27., 26., 25.])

In [53]:
# plt.figure()
# plt.plot(efp_array.coord('latitude').points, efp_array.data)
# plt.show()

# QG Eddy Feedback Parameter

In [54]:
U = ds.u.to_iris()
V = ds.v.to_iris()

U.units = 'm s-1'
V.units = 'm s-1'

V

<iris 'Cube' of northward_wind / (m s-1) (time: 4030; air_pressure: 37; latitude: 181; longitude: 360)>

In [55]:
divFh = saff.horiz_EPflux_div(U,V) 
Ubar = ds.ubar.to_iris() 

/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'longitude', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'longitude'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'longitude', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'longitude'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'longitude', with 0 bound(s). Contigu

In [56]:
## ERA5
divFh = divFh.extract(iris.Constraint(air_pressure=500))
Ubar = Ubar.extract(iris.Constraint(air_pressure=500)) 


## JRA55
# divFh = divFh.extract(iris.Constraint(level=500))
# Ubar = Ubar.extract(iris.Constraint(level=500)) 

In [57]:
divFh = saff.seasonal_mean(divFh, span)
Ubar = saff.seasonal_mean(Ubar, span) 

In [58]:
EFP, EFP_array = saff.eddy_feedback_parameter(divFh, Ubar)

/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2245: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'season_year', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'season_year'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))
/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2245: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/emmy-noether/home/ct715/miniforge3/envs/eddy/lib/python3.12/site-packages/iris/coords.py:2237: UserWarning: Cannot check

In [59]:
EFP.data 

array(0.28156312)

In [60]:
EFP_array.data

masked_array(data=[0.13772913, 0.17719996, 0.22326611, 0.29296564,
                   0.32771686, 0.35851049, 0.41316065, 0.44538776,
                   0.41205465, 0.40336606, 0.42451658, 0.48945449,
                   0.50713055, 0.4286717 , 0.43676311, 0.3937158 ,
                   0.32761273, 0.35986538, 0.33663859, 0.35452897,
                   0.35430424, 0.34256938, 0.41771872, 0.46701597,
                   0.47638857, 0.43273053, 0.42827898, 0.40034829,
                   0.42746503, 0.38529847, 0.28152217, 0.16257338,
                   0.06184123, 0.07646309, 0.064049  , 0.0166099 ,
                   0.01338026, 0.06993937, 0.09060255, 0.05126733,
                   0.01715977, 0.03310801, 0.240157  , 0.41359405,
                   0.41084242, 0.2615983 , 0.26680747, 0.37996322],
             mask=False,
       fill_value=1e+20)

In [61]:
# plt.figure()
# plt.plot(EFP_array.coord('latitude').points, EFP_array.data)
# plt.show()

In [62]:
# fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,5))

# ax1.plot(efp_array.coord('latitude').points, efp_array.data)
# ax1.set_xlabel('Lat')
# ax1.set_ylabel('Corr$^2$(div1, ubar)')
# ax1.set_title('Full primitive (calc using aostools)')

# ax2.plot(EFP_array.coord('latitude').points, EFP_array.data)
# ax2.set_xlabel('Lat')
# ax2.set_ylabel('Corr$^2$(div1, ubar)')
# ax2.set_title('QG version (calc using Saffin code)')

# plt.show()

# QG using aostools

In [63]:
ds = xr.open_mfdataset('/home/links/ct715/eddy_feedback/daily_datasets/jra55_djf_noAttrs.nc', 
                       parallel=True, chunks={'time':31})

ds

<xarray.Dataset>
Dimensions:    (time: 5325, bnds: 2, lon: 144, lat: 73, lev: 37, lev_2: 37)
Coordinates:
  * time       (time) datetime64[ns] 1958-01-01T09:00:00 ... 2016-12-31T09:00:00
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat        (lat) float64 90.0 87.5 85.0 82.5 ... -82.5 -85.0 -87.5 -90.0
  * lev        (lev) float64 1e+05 9.75e+04 9.5e+04 ... 300.0 200.0 100.0
  * lev_2      (lev_2) float64 100.0 200.0 300.0 ... 9.5e+04 9.75e+04 1e+05
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(31, 2), meta=np.ndarray>
    u          (time, lev, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    v          (time, lev_2, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    t          (time, lev_2, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>
    omega      (time, lev_2, lat, lon) float32 dask.array<chunksize=(31, 37, 73, 144), meta=np.ndarray>

In [64]:
import sys
sys.path.append('/home/links/ct715/eddy_feedback')
import functions.eddy_feedback as ef 

In [65]:
ds = ef.calculate_epfluxes_ubar(ds, primitive=False)
ds

Variables are already named as required.


KeyError: 'level'